# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
T. Henning  ->  T. Henning  |  ['T. Henning']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']
S. Barboza  ->  S. Barboza  |  ['S. Barboza']
P. Bizenberger  ->  P. Bizenberger  |  ['P. Bizenberger']
A. Boné  ->  A. Boné  |  ['A. Boné']
R. Hofferbert  ->  R. Hofferbert  |  ['R. Hofferbert']
J. Jost  ->  J. Jost  |  ['J. Jost']
F. Lang  ->  F. Lang  |  ['F. Lang']
L. Mohr  ->  L. Mohr  |  ['L. Mohr']
D. Muley  ->  D. Muley  |  ['D. Muley']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
Arxiv has 61 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.16053


extracting tarball to tmp_2408.16053...

 done.


K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']


...

/tmp/ipykernel_2646/1211882699.py:51: LatexWarning: 2408.16396 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2408.16461...

 done.


D. Muley  ->  D. Muley  |  ['D. Muley']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']


Found 63 bibliographic references in tmp_2408.16461/threetemp_spiral_aanda_rev1.bbl.
syntax error in line 441: premature end of file


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.16053-b31b1b.svg)](https://arxiv.org/abs/2408.16053) | **Cataclysmic Variables and AM CVn Binaries in SRG/eROSITA + Gaia: Volume Limited Samples, X-ray Luminosity Functions, and Space Densities**  |
|| A. C. Rodriguez, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-08-30*|
|*Comments*| *Submitted to PASP, comments welcome*|
|**Abstract**|            We present volume-limited samples of cataclysmic variables (CVs) and AM CVn binaries jointly selected from SRG/eROSITA eRASS1 and \textit{Gaia} DR3 using an X-ray + optical color-color diagram (the ``X-ray Main Sequence"). This tool identifies all CV subtypes, including magnetic and low-accretion rate systems, in contrast to most previous surveys. We find 23 CVs, 3 of which are AM CVns, out to 150 pc in the Western Galactic Hemisphere. Our 150 pc sample is spectroscopically verified and complete down to $L_X = 1.3\times 10^{29} \;\textrm{erg s}^{-1}$ in the 0.2--2.3 keV band, and we also present CV candidates out to 300 pc and 1000 pc. We discovered two previously unknown systems in our 150 pc sample: the third nearest AM CVn and a magnetic period bouncer. We find the mean $L_X$ of CVs to be $\langle L_X \rangle \approx 4.6\times 10^{30} \;\textrm{erg s}^{-1}$, in contrast to previous surveys which yielded $\langle L_X \rangle \sim 10^{31}-10^{32} \;\textrm{erg s}^{-1}$. We construct X-ray luminosity functions that, for the first time, flatten out at $L_X\sim 10^{30} \; \textrm{erg s}^{-1}$. We find average number, mass, and luminosity densities of $\rho_\textrm{N, CV} = (3.7 \pm 0.7) \times 10^{-6} \textrm{pc}^{-3}$, $\rho_M = (5.0 \pm 1.0) \times 10^{-5} M_\odot^{-1}$, and $\rho_{L_X} = (2.3 \pm 0.4) \times 10^{26} \textrm{erg s}^{-1}M_\odot^{-1}$, respectively, in the solar neighborhood. Our uniform selection method also allows us to place meaningful estimates on the space density of AM CVns, $\rho_\textrm{N, AM CVn} = (5.5 \pm 3.7) \times 10^{-7} \textrm{pc}^{-3}$. Magnetic CVs and period bouncers make up $35\%$ and $25\%$ of our sample, respectively. This work, through a novel discovery technique, shows that the observed number densities of CVs and AM CVns, as well as the fraction of period bouncers, are still in tension with population synthesis estimates.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.16367-b31b1b.svg)](https://arxiv.org/abs/2408.16367) | **Dust mineralogy and variability of the inner PDS 70 disk**  |
|| H. Jang, et al. -- incl., <mark>G. Perotti</mark>, <mark>T. Henning</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2024-08-30*|
|*Comments*| *18 pages, 13 figures, Accepted by A&A*|
|**Abstract**|            The inner disk of the young star PDS 70 may be a site of rocky planet formation, with two giant planets detected further out. Solids in the inner disk may inform us about the origin of this inner disk water and nature of the dust in the rocky planet-forming regions. We aim to constrain the chemical composition, lattice structure, and grain sizes of small silicate grains in the inner disk of PDS 70, observed both in JWST/MIRI MRS and Spitzer IRS. We use a dust fitting model, called DuCK, based on a two-layer disk model. We use Gaussian Random Field and Distribution of Hollow Spheres models to obtain two sets of dust opacities. The third set of opacities is obtained from aerosol spectroscopy. We use stoichiometric amorphous silicates, forsterite, and enstatite in our analysis. We also used iron-rich and magnesium-rich amorphous silicate and fayalite dust species to study the iron content. The Gaussian Random Field opacity agrees well with the observed spectrum. In both MIRI and Spitzer spectra, amorphous silicates are the dominant dust species. Crystalline silicates are dominated by iron-poor olivine. We do not find strong evidence for enstatite. Moreover, the MIRI spectrum indicates larger grain sizes than the Spitzer spectrum, indicating a time-variable small grain reservoir. The inner PDS 70 disk is dominated by a variable reservoir of optically thin warm amorphous silicates. We suggest that the small grains detected in the inner PDS 70 disk are likely transported inward from the outer disk as a result of filtration and fragmentation at the ice line. In addition, the variation between MIRI and Spitzer data can be explained by the grain growth over 15 years and a dynamical inner disk where opacity changes occur resulting from the highly variable hot innermost dust reservoir.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.16461-b31b1b.svg)](https://arxiv.org/abs/2408.16461) | **Can gap-edge illumination excite spirals in protoplanetary disks? Three-temperature radiation hydrodynamics and NIR image modelling**  |
|| <mark>D. Muley</mark>, J. D. M. Fuksman, <mark>H. Klahr</mark> |
|*Appeared on*| *2024-08-30*|
|*Comments*| *11 pages, 9 figures incl. Appendices. A&A accepted; further comments and questions welcome*|
|**Abstract**|            The advent of high-resolution, near-infrared instruments such as VLT/SPHERE and Gemini/GPI has helped uncover a wealth of substructure in planet-forming disks, including large, prominent spiral arms in MWC 758, SAO 206462, and V1247 Ori among others. In the classical theory of disk-planet interaction, these arms are consistent with Lindblad-resonance driving by multi-Jupiter-mass companions. Despite improving detection limits, evidence for such massive bodies in connection with spiral substructure has been inconclusive. In search of an alternative explanation, we use the PLUTO code to run 3D hydrodynamical simulations with two comparatively low planet masses (Saturn-mass, Jupiter-mass) and two thermodynamic prescriptions (three-temperature radiation hydrodynamics, and the more traditional $\beta$-cooling) in a low-mass disk. In the radiative cases, an $m = 2$ mode, potentially attributable to the interaction of stellar radiation with gap-edge asymmetries, creates an azimuthal pressure gradient, which in turn gives rise to prominent spiral arms in the upper layers of the disk. Monte Carlo radiative transfer (MCRT) post-processing with RADMC3D reveals that in near-infrared scattered light, these gap-edge spirals are significantly more prominent than the traditional Lindblad spirals for planets in the mass range tested. Our results demonstrate that even intermediate-mass protoplanets -- less detectable, but more ubiquitous, than super-Jupiters -- are capable of indirectly inducing large-scale spiral disk features, and underscore the importance of including radiation physics in efforts to reproduce observations.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.16396-b31b1b.svg)](https://arxiv.org/abs/2408.16396) | **The MICADO first light imager for the ELT: overview and current Status**  |
|| E. Sturm, et al. -- incl., <mark>J.-U. Pott</mark>, <mark>S. Barboza</mark>, <mark>P. Bizenberger</mark>, <mark>A. Boné</mark>, <mark>R. Hofferbert</mark>, <mark>J. Jost</mark>, <mark>F. Lang</mark>, <mark>L. Mohr</mark> |
|*Appeared on*| *2024-08-30*|
|*Comments*| *Proceedings of the SPIE, Volume 13096, id. 1309611 11 pp. (2024)*|
|**Abstract**|            MICADO is a first light instrument for the Extremely Large Telescope (ELT), set to start operating later this decade. It will provide diffraction limited imaging, astrometry, high contrast imaging, and long slit spectroscopy at near-infrared wavelengths. During the initial phase operations, adaptive optics (AO) correction will be provided by its own natural guide star wavefront sensor. In its final configuration, that AO system will be retained and complemented by the laser guide star multi-conjugate adaptive optics module MORFEO (formerly known as MAORY). Among many other things, MICADO will study exoplanets, distant galaxies and stars, and investigate black holes, such as Sagittarius A* at the centre of the Milky Way. After their final design phase, most components of MICADO have moved on to the manufacturing and assembly phase. Here we summarize the final design of the instrument and provide an overview about its current manufacturing status and the timeline. Some lessons learned from the final design review process will be presented in order to help future instrumentation projects to cope with the challenges arising from the substantial differences between projects for 8-10m class telescopes (e.g. ESO-VLT) and the next generation Extremely Large Telescopes (e.g. ESO-ELT). Finally, the expected performance will be discussed in the context of the current landscape of astronomical observatories and instruments. For instance, MICADO will have similar sensitivity as the James Webb Space Telescope (JWST), but with six times the spatial resolution.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.16053/./all_sys1.png', 'tmp_2408.16053/./all_sys2.png', 'tmp_2408.16053/./periodbouncer.png', 'tmp_2408.16053/./lum_dist.png']
copying  tmp_2408.16053/./all_sys1.png to _build/html/
copying  tmp_2408.16053/./all_sys2.png to _build/html/
copying  tmp_2408.16053/./periodbouncer.png to _build/html/
copying  tmp_2408.16053/./lum_dist.png to _build/html/
exported in  _build/html/2408.16053.md
    + _build/html/tmp_2408.16053/./all_sys1.png
    + _build/html/tmp_2408.16053/./all_sys2.png
    + _build/html/tmp_2408.16053/./periodbouncer.png
    + _build/html/tmp_2408.16053/./lum_dist.png
found figures ['tmp_2408.16367/./figures/MIRI_Spitzer_SpectraF98F113.png', 'tmp_2408.16367/./figures/QcurvesCompare.png', 'tmp_2408.16367/./figures/temperatures_vertical.png']
copying  tmp_2408.16367/./figures/MIRI_Spitzer_SpectraF98F113.png to _build/html/
copying  tmp_2408.16367/./figures/QcurvesCompare.png to _build/html/
copying  tmp_2408.16367/./figures/temperatures_vertical.png 

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Cataclysmic Variables and AM CVn Binaries in SRG/eROSITA + _Gaia_: Volume Limited Samples, X-ray Luminosity Functions, and Space Densities

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.16053-b31b1b.svg)](https://arxiv.org/abs/2408.16053)<mark>Appeared on: 2024-08-30</mark> -  _Submitted to PASP, comments welcome_

</div>
<div id="authors">

A. C. Rodriguez, et al. -- incl., <mark>K. El-Badry</mark>

</div>
<div id="abstract">

**Abstract:** We present volume-limited samples of cataclysmic variables (CVs) and AM CVn binaries jointly selected from SRG/eROSITA eRASS1 and _Gaia_ DR3 using an X-ray + optical color-color diagram (the "X-ray Main Sequence"). This tool identifies all CV subtypes, including magnetic and low-accretion rate systems, in contrast to most previous surveys. We find 23 CVs, 3 of which are AM CVns, out to 150 pc in the Western Galactic Hemisphere. Our 150 pc sample is spectroscopically verified and complete down to $L_X = 1.3\times 10^{29} \;\textrm{erg s}^{-1}$ in the 0.2--2.3 keV band, and we also present CV candidates out to 300 pc and 1000 pc. We discovered two previously unknown systems in our 150 pc sample: the third nearest AM CVn and a magnetic period bouncer. We find the mean $L_X$ of CVs to be $\langle L_X \rangle \approx 4.6\times 10^{30} \;\textrm{erg s}^{-1}$ , in contrast to previous surveys which yielded $\langle L_X \rangle \sim 10^{31}-10^{32} \;\textrm{erg s}^{-1}$ .  We construct X-ray luminosity functions that, for the first time, flatten out at $L_X\sim 10^{30} \;\textrm{erg s}^{-1}$ . We find average number, mass, and luminosity densities of $\rho_\textrm{N, CV} = (3.7 \pm 0.7) \times 10^{-6} \textrm{pc}^{-3}$ , $\rho_M = (5.0 \pm 1.0) \times 10^{-5} M_\odot^{-1}$ , and $\rho_{L_X} = (2.3 \pm 0.4) \times 10^{26} \textrm{erg s}^{-1}M_\odot^{-1}$ , respectively, in the solar neighborhood. Our uniform selection method also allows us to place meaningful estimates on the space density of AM CVns, $\rho_\textrm{N, AM CVn} = (5.5 \pm 3.7) \times 10^{-7} \textrm{pc}^{-3}$ . Magnetic CVs and period bouncers make up $35\%$ and $25\%$ of our sample, respectively. This work, through a novel discovery technique, shows that the observed number densities of CVs and AM CVns, as well as the fraction of period bouncers, are still in tension with population synthesis estimates.

</div>

<div id="div_fig1">

<img src="tmp_2408.16053/./all_sys1.png" alt="Fig8.1" width="50%"/><img src="tmp_2408.16053/./all_sys2.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Our 150 pc sample in the X-ray Main Sequence (upper left, with all systems in our 150 pc crossmatch in black), _Gaia_ HR diagram (lower left, with the 100 pc _Gaia_ catalog in black), and in the $P_\textrm{orb}$--$L_X$ parameter space (right, with the VSX + eROSITA CV sample in gray). Theoretical $L_X$ values are plotted as a function of period, using donor mass loss estimates from the CV evolutionary tracks ("standard" in gray and "optimal" in black) of [Pala, Gänsicke and Townsley (2017)](),  "He WD progenitor"/"evolved CV progenitor" AM CVn tracks of [Wong and Bildsten (2021)]()/[Sarkar, Ge and Tout (2023)]() and Equation 1. Accretion efficiencies in the range of $\eta \approx 0.02-0.3$(dashed and solid tracks) best fit the data, which could be confirmed through X-ray spectral analyses that likely depend on CV subclass. Observations of trends in these parameter spaces by CV subclass are presented in Table \ref{tab:all}. (*fig:all*)

</div>
<div id="div_fig2">

<img src="tmp_2408.16053/./periodbouncer.png" alt="Fig11" width="100%"/>

**Figure 11. -** 1eRASS J054726.9+132649 is a newly-discovered magnetic period bouncer, selected thanks to the X-ray Main Sequence (upper left). It is located near the WD track (bottom left), and shows no outbursts in six years of data (bottom center), indicative of low mass transfer rates. ZTF optical photometry ($r$-band in red, $g$-band in blue) reveals a 1.57 hr orbital period (bottom right). The optical spectrum (upper right) shows that a 2900 K donor star predicted by CV evolutionary tracks (see Figure \ref{fig:pb_proof}) at this orbital period is not seen, thus confirming this system as a period bouncer. Balmer emission lines show clear Zeeman splitting (particularly H$\beta$ and H$\gamma$), which reveal the magnetic nature of the WD.  (*fig:pb*)

</div>
<div id="div_fig3">

<img src="tmp_2408.16053/./lum_dist.png" alt="Fig2" width="100%"/>

**Figure 2. -** Observed $L_X$ distributions of CVs. $L_X$ distributions of primarily optically-identified systems that have eROSITA (cyan) and ROSAT (blue) X-ray counterparts suggest that CVs have $\langle L_X\rangle \sim 10^{31-32}\textrm{erg s}^{-1}$. Our 150 pc volume limited sample of systems selected using the X-ray Main Sequence (red) instead reveals that $\langle L_X \rangle \sim 10^{30}\textrm{erg s}^{-1}$. (*fig:lum_dist*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.16053"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sect}[1]{Sect.~\ref{sec:#1}}$
$\newcommand{\Sect}[1]{Section~\ref{sec:#1}}$
$\newcommand{\sects}[2]{Sects. \ref{sec:#1} and \ref{sec:#2}}$
$\newcommand{\Sects}[2]{Sections \ref{sec:#1} and \ref{sec:#2}}$
$\newcommand{\fg}[1]{Fig.~\ref{fig:#1}}$
$\newcommand{\Fg}[1]{Figure~\ref{fig:#1}}$
$\newcommand{\fgs}[2]{Figs. \ref{fig:#1} and \ref{fig:#2}}$
$\newcommand{\Fgs}[2]{Figures \ref{fig:#1} and \ref{fig:#2}}$
$\newcommand{\fgnum}[1]{\ref{fig:#1}}$
$\newcommand{\eq}[1]{Eq.~(\ref{eq:#1})\xspace}$
$\newcommand{\Eq}[1]{Equation~(\ref{eq:#1})\xspace}$
$\newcommand{\Eqs}[2]{Eqs. (\ref{eq:#1}) and (\ref{eq:#2})}$
$\newcommand{\Eqss}[3]{Eqs. (\ref{eq:#1}), (\ref{eq:#2}) and (\ref{eq:#3})}$
$\newcommand{\co}[1]{\textcolor{blue}{\textsf{#1}}}$
$\newcommand{\jay}[1]{\textcolor{orange}{\textsf{#1}}}$</div>



<div id="title">

# Dust mineralogy and variability of the inner PDS 70 disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.16367-b31b1b.svg)](https://arxiv.org/abs/2408.16367)<mark>Appeared on: 2024-08-30</mark> -  _18 pages, 13 figures, Accepted by A&A_

</div>
<div id="authors">

H. Jang, et al. -- incl., <mark>G. Perotti</mark>, <mark>T. Henning</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** _Context._ The inner disk of the young star PDS 70 may be a site of rocky planet formation, with two giant planets detected further out. Recently, JWST/MIRI MRS observations have revealed the presence of warm water vapour in the inner disk. Solids in the inner disk may inform us about the origin of this inner disk water and nature of the dust in the rocky planet-forming regions of the disk. \ _Aims._ We aim to constrain the chemical composition, lattice structure, and grain sizes of small silicate grains in the inner disk of PDS 70, observed both in JWST/MIRI MRS and Spitzer IRS. \ _Methods._ We use a dust fitting model, called DuCK, based on a two-layer disk model considering three different sets of dust opacities. We use Gaussian Random Field and Distribution of Hollow Spheres models to obtain two sets of dust opacities using the optical constants of cosmic dust analogues derived from laboratory-based measurements. These sets take into account the grain sizes as well as their shapes. The third set of opacities is obtained from the experimentally measured transmission spectra from aerosol spectroscopy. We use stoichiometric amorphous silicates, forsterite, and enstatite in our analysis. We also study the iron content of crystalline olivine using the resonance at 23-24 $\mu$ m and test the presence of fayalite. Both iron-rich and magnesium-rich amorphous silicate dust species are also employed to fit the observed spectra. \ _Results._ The Gaussian Random Field opacity set agrees well with the observed spectrum, better than the other two opacity sets. In both MIRI and Spitzer spectra, amorphous silicates are the dominant dust species. Crystalline silicates are dominated by iron-poor olivine. The 23-24 $\mu$ m olivine band peaks at 23.44 $\mu$ m for the MIRI spectrum and 23.47 $\mu$ m for the Spitzer spectrum, representing around or less than 10 \% of iron content in the crystalline silicate. In all of models, we do not find strong evidence for enstatite. Moreover, the silicate band in the MIRI spectrum indicates larger grain sizes (a few microns up to 5 $\mu$ m) than the Spitzer spectrum (0.1 to 1 $\mu$ m), indicating a time-variable small grain reservoir. \ _Conclusions._ The inner PDS 70 disk is dominated by a variable reservoir of warm (T $\sim$ 350-500 K) amorphous silicates, with $\sim$ 15 \% of forsterite in mass fraction. The 10 $\mu$ m and 18 $\mu$ m amorphous silicate bands are very prominent, indicating that most emission originates from optically thin dust. We suggest that the small grains detected in the PDS 70 inner disk are likely transported inward from the outer disk as a result of filtration by the pressure bump associated with the gap and fragmentation into smaller sizes at the ice line. Collisions among larger parent bodies may also contribute to the small grain reservoir in the inner disk, but these parent bodies must be enstatite-poor. In addition, the variation between MIRI and Spitzer data can be explained by a combination of grain growth over 15 years and a dynamical inner disk where opacity changes occur resulting from the highly variable hot (T $\sim$ 1000 K) innermost dust reservoir.

</div>

<div id="div_fig1">

<img src="tmp_2408.16367/./figures/MIRI_Spitzer_SpectraF98F113.png" alt="Fig1" width="100%"/>

**Figure 1. -** The MIRI and Spitzer spectra of PDS 70 and their shapes of 10 $\mu$m silicate bands. _Upper_: MIRI spectrum is in blue, and Spitzer spectrum is in orange. Light orange shows full wavelength range of the Spitzer spectrum. Around $10 \mu$m, the MIRI spectrum has a higher flux and broader silicate band while the Spitzer spectrum shows higher flux levels beyond $12 \mu$m. _Lower_: Band strength at 9.8 $\mu$m and the shape of 10 $\mu$m silicate band. Gray dots are disks in CASSIS database with low resolution spectra. Blue cross is the MIRI spectrum while orange cross is the Spitzer spectrum. (*fig:MIRI_Spitzer*)

</div>
<div id="div_fig2">

<img src="tmp_2408.16367/./figures/QcurvesCompare.png" alt="Fig2" width="100%"/>

**Figure 2. -** Comparison among absorption efficiencies for amorphous $Mg_2$$SiO_4$(upper panel) and forsterite (lower panel). GRF (black solid line) and DHS (dashed line) are plotted for $Q_{\rm abs}$ on the left y-axis, and Aerosol spectra (gray solid line) are normalized on the right y-axis. (*fig:Qcompare*)

</div>
<div id="div_fig3">

<img src="tmp_2408.16367/./figures/temperatures_vertical.png" alt="Fig3" width="100%"/>

**Figure 3. -** Temperatures of dust in the inner rim (top), disk surface (middle), and midplane (bottom) of dust fitting models in Table \ref{tab:results}. For Spitzer(full), the dotted line in the temperature of the disk surface indicates the temperature gradient across the disk surface; the other models assume a single temperature. (*fig:temperatures*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.16367"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\dmnote}[1]{{\bf \color{blue}[D.M.: #1]}}$
$\newcommand{\dmfnote}[1]{{\bf \color{orange}[D.M.F.: #1]}}$
$\newcommand{\hknote}[1]{{\bf \color{green}[H.K.: #1]}}$
$\newcommand{\revision}[1]{{#1}}$</div>



<div id="title">

#  Can gap-edge illumination excite spirals in protoplanetary disks?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.16461-b31b1b.svg)](https://arxiv.org/abs/2408.16461)<mark>Appeared on: 2024-08-30</mark> -  _11 pages, 9 figures incl. Appendices. A&A accepted; further comments and questions welcome_

</div>
<div id="authors">

<mark>D. Muley</mark>, J. D. M. Fuksman, <mark>H. Klahr</mark>

</div>
<div id="abstract">

**Abstract:** The advent of high-resolution, near-infrared instruments such as VLT/SPHERE and Gemini/GPI has helped uncover a wealth of substructure in planet-forming disks, including large, prominent spiral arms in MWC 758, SAO 206462, and V1247 Ori among others. In the classical theory of disk-planet interaction, these arms are consistent with Lindblad-resonance driving by multi-Jupiter-mass companions. Despite improving detection limits, evidence for such massive bodies in connection with spiral substructure has been inconclusive. In search of an alternative explanation, we use the PLUTO code to run 3D hydrodynamical simulations with two comparatively low planet masses (Saturn-mass, Jupiter-mass) and two thermodynamic prescriptions (three-temperature radiation hydrodynamics, and the more traditional $\beta$ -cooling) in a low-mass disk. In the radiative cases, an $m = 2$ mode, potentially attributable to the interaction of stellar radiation with gap-edge asymmetries, creates an azimuthal pressure gradient, which in turn gives rise to prominent spiral arms $\revision{in the upper layers of the disk}$ . Monte Carlo radiative transfer (MCRT) post-processing with RADMC3D reveals that $\revision{in near-infrared scattered light, }$ these $\revision{gap-edge}$ spirals are significantly more prominent than $\revision{the}$ traditional Lindblad spirals $\revision{for planets in the mass range tested}$ . Our results demonstrate that even intermediate-mass **proto** planets---less detectable, but more ubiquitous, than super-Jupiters---are capable of indirectly inducing large-scale spiral disk features, and underscore the importance of including radiation physics in efforts to reproduce observations.

</div>

<div id="div_fig1">

<img src="tmp_2408.16461/./dens_compare.png" alt="Fig7" width="100%"/>

**Figure 7. -** Gas density $\rho$ at $t = 1010$ orbits (1000 orbits at the fiducial resolution, 10 at the doubled resolution used in \cite{Muley2024}), with respect to the initial condition $\rho_0$, for our simulations with a Saturn-mass planet. The green ellipse indicates the planet's Hill radius. With the 3T scheme, the disk atmosphere shows clear development of $m = 2$ spiral arms, which are absent in the $\beta$-cooling simulations. $\revision${White bands at the interior and exterior of the radiative simulation result from wave-damping to the initial condition.} (*fig:dens-compare*)

</div>
<div id="div_fig2">

<img src="tmp_2408.16461/./initial_conditions_upd2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Initial conditions for gas density $\rho$(above), $\revision${as well as dust and gas temperatures $T_d$ and $T_g$ (below), which are initially equal.} Contours for normalized cooling timescales from gas-grain collision ($\beta_{\rm dg}$) are plotted above, and from radiation ($\beta_{\rm rad}$) below, with white lines. Figure reproduced exactly from \cite{Muley2024}. (*fig:initial_conditions*)

</div>
<div id="div_fig3">

<img src="tmp_2408.16461/./azimuthal_profiles.png" alt="Fig2" width="100%"/>

**Figure 2. -** Azimuthal profiles of various quantities at a fiducial radius of $r = 60 {\rm  au} = 1.5 r_p$, and at an altitude of $\theta = 0.2$ rad above the midplane $\revision${in our $M_p = 3 \times 10^{-4} M_{\odot}$ simulations}. In the 3T case, azimuthal asymmetries of even a few percent in disk illumination---potentially induced by the RWI, and visible in the $T_d$ profile---lead to strong non-axisymmetry in $\rho$, $T_g$, and the velocity components.
     (*fig:azi_profiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.16461"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

171  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
